# 불법주정차 데이터 위도/경도 수집 (Kakao Api)


### Present working directory

In [12]:
!pwd

/Users/lyuhoyun/Documents/GitHub/Data-projects/dacon_sri_suwon_data_analysis


### 패키지 설치

In [11]:
!pip install pandas numpy tqdm requests

### 패키지 다운로드

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import time
import json
import requests

### 카카오 API키 및 API 동작확인

In [ ]:
api_key = "API KEY"

In [14]:
addr = '서울시 송파구 위례광장로 185'
url = 'https://dapi.kakao.com/v2/local/search/address.json?query={address}'.format(address=addr)
headers = {
    "Authorization": f"KakaoAK {api_key}"
}
params = {"query": addr}

result = json.loads(str(requests.get(url, headers=headers).text))
display(result)

{'documents': [{'address': {'address_name': '서울 송파구 장지동 878',
    'b_code': '1171010900',
    'h_code': '1171064700',
    'main_address_no': '878',
    'mountain_yn': 'N',
    'region_1depth_name': '서울',
    'region_2depth_name': '송파구',
    'region_3depth_h_name': '위례동',
    'region_3depth_name': '장지동',
    'sub_address_no': '',
    'x': '127.14004016241',
    'y': '37.4804774688545'},
   'address_name': '서울 송파구 위례광장로 185',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '서울 송파구 위례광장로 185',
    'building_name': '위례신도시 송파푸르지오',
    'main_building_no': '185',
    'region_1depth_name': '서울',
    'region_2depth_name': '송파구',
    'region_3depth_name': '장지동',
    'road_name': '위례광장로',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '127.14004016241',
    'y': '37.4804774688545',
    'zone_no': '05848'},
   'x': '127.14004016241',
   'y': '37.4804774688545'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

### 데이터 불러오기

In [4]:
data1 = pd.read_csv('parking_notcctv_1.csv')
data2 = pd.read_csv('parking_notcctv_2.csv')
data3 = pd.read_csv('parking_notcctv_3.csv')
data4 = pd.read_csv('parking_notcctv_4.csv')
data5 = pd.read_csv('parking_notcctv_5.csv')

### 데이터 수집 및 적용 함수

In [ ]:
def get_lat_lng(addr):
    headers = {
        "Authorization": f"KakaoAK {api_key}"
    }
    params = {"query": addr}
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    
    response = requests.get(url, headers=headers, params=params)
    time.sleep(0.15)
    if response.status_code == 200:
        documents = response.json().get("documents")
        if documents:
            return float(documents[0]["y"]), float(documents[0]["x"])  # 위도, 경도
    return None, None


def add_to_pos(data):
    tqdm.pandas()

    latitudes = []
    longitudes = []

    for _, row in tqdm(data.iterrows(), total = len(data)):
        lat, lng = get_lat_lng(row['단속장소'])
        latitudes.append(lat)
        longitudes.append(lng)

    data['latitude'] = latitudes
    data['longitude'] = longitudes

    display(data.head(10))

In [ ]:
#샘플 테스트
sample=data1.sample(10)

### 적용

In [8]:
add_to_pos(sample)

100%|██████████| 10/10 [00:01<00:00,  5.87it/s]


,집계년도,시군명,관리기관명,단속일시정보,단속장소,단속방법,데이터기준일자,단속일시_월,단속일시_연,단속일시_일,단속일시_요일,단속일시_시,단속일시_isdaytime,단속일시_season,latitude,longitude
24802,2022,수원시,영통구청,2022-02-22 19:52:00,이의동 센트럴타운로,주행형,2023-12-01,2,2022,22,Tuesday,19,night,winter,37.290024,127.053432
13454,2022,수원시,영통구청,2022-01-27 14:22:00,원천동 월드컵로193번길,주행형,2023-12-01,1,2022,27,Thursday,14,afternoon,winter,37.277262,127.044756
17861,2022,수원시,권선구청,2022-02-08 20:12:00,호매실동 호매실로104번길,주행형,2023-12-01,2,2022,8,Tuesday,20,night,winter,37.258327,126.962288
71998,2022,수원시,영통구청,2022-05-18 19:54:00,매탄동 매탄로126번길,주행형,2023-12-01,5,2022,18,Wednesday,19,night,spring,37.261470,127.046333
53743,2022,수원시,장안구청,2022-04-16 10:29:00,하광교동 수원북부순환로,주행형,2023-12-01,4,2022,16,Saturday,10,morning,spring,NaN,NaN
30729,2022,수원시,권선구청,2022-03-07 09:19:00,금곡동 서수원로577번길,주행형,2023-12-01,3,2022,7,Monday,9,morning,spring,37.276814,126.946092
57752,2022,수원시,팔달구청,2022-04-22 19:29:00,화서동 184-97 (교차로모퉁이 5m 이내),국민신문고,2023-12-01,4,2022,22,Friday,19,night,spring,37.277935,127.003397
81807,2022,수원시,영통구청,2022-06-06 15:56:00,신동 권선로908번길,주행형,2023-12-01,6,2022,6,Monday,15,afternoon,summer,37.247108,127.044514
41701,2022,수원시,영통구청,2022-03-26 13:15:00,영통구 영통동 998-6,국민신문고,2023-12-01,3,2022,26,Saturday,13,afternoon,spring,37.252069,127.074449
43638,2022,수원시,영통구청,2022-03-30 14:31:00,망포동 영통로153번길,주행형,2023-12-01,3,2022,30,Wednesday,14,afternoon,spring,37.241545,127.052535


### 실제 데이터 주소 -> 위/경도 전환 
시간 소요될 수 있음. 요청 당 0.15초 딜레이

In [ ]:
partial_data = data1

In [ ]:
add_to_pos(partial_data)